In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import numba as nb

In [4]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)
print(f'{indices} \n {distances}')

[[0 1]
 [1 0]
 [2 1]
 [3 4]
 [4 3]
 [5 4]] 
 [[0.         1.        ]
 [0.         1.        ]
 [0.         1.41421356]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.41421356]]
